# Simple Linear Regression
19/09/2022 by LimBus

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD
import torch.nn as nn
import torch
import time
import matplotlib.pyplot as plt

## Construct dataset

In [ ]:
class ToyDataset1D(Dataset):
  def __init__(self, lenght=100):
    self.len = lenght
    self.X = torch.linspace(-1,1,lenght)
    rand = 2*torch.rand_like(self.X) - 1
    self.y = 2*self.X + 0.5*rand
  
  def __getitem__(self, index):
    sample = self.X[index], self.y[index]
    return sample
  
  def __len__(self):
    return self.len

In [ ]:
mydata = ToyDataset1D()
X, y = mydata[:]
plt.scatter(X,y)
plt.show()

[`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

In [ ]:
trainloader = DataLoader(dataset=mydata, batch_size=1)

## Construct model

In [ ]:
torch.manual_seed(1)
class LinearRegression(nn.Module):
  def __init__(self, input_nodes, output_nodes, bias=False):
    super(LinearRegression, self).__init__()
    self.linear = torch.nn.Linear(input_nodes, output_nodes, bias=bias)
    # self.linear.weight.detach().zero_()

  def forward(self, x):
    return self.linear(x)

In [ ]:
model = LinearRegression(1,1,False)
model

In [ ]:
from torchsummary import summary
summary(model, X.view(-1,1).shape)

## Loss & Hyperparameters setting

In [ ]:
loss_fn = nn.MSELoss()
epochs = 10
learning_rate = .1
COST = []

## Construct optimizer

In [ ]:
optimizer = SGD(model.parameters(), lr=learning_rate)
optimizer

## Training model

In [ ]:
start_time = time.time()
for epoch in range(epochs):
  losses_ = []
  for X, y in trainloader:
    X, y = X.view(-1,1), y.view(-1,1)
    model.train()
    #### Compute outputs ####
    output = model(X)
    
    #### Compute Loss ####
    loss  = loss_fn(output, y)
    losses_.append(loss.item())

    #### Compute gradients ####
    optimizer.zero_grad()
    loss.backward()
    
    #### Update parameters ####  
    optimizer.step()
  
  COST.append(sum(losses_)/len(losses_))

  ### Logging ###
  print('Epoch: %03d/%03d' % ((epoch + 1), epochs), end="")
  print(' | Loss: %.3f \n' % COST[-1], end="")

print('weight: %s' % model.linear.weight.item())
if model.linear.bias:
  print('bias: %s' % model.linear.bias.item())
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
# [optional] save model
checkpoint_dict = {
    'model_state_dict' : model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'cost' : COST
    }
torch.save(checkpoint_dict, '/content/mymodel.pth')

## Evaluation model

In [ ]:
checkpoint = torch.load('/content/mymodel.pth', map_location='cpu')
checkpoint

In [ ]:
mydata = ToyDataset1D(50)
X, y = mydata[:]

model = LinearRegression(1,1,False)
model.load_state_dict(checkpoint['model_state_dict'])
COST = checkpoint['cost']

model.eval()
X_eval = torch.linspace(-1,1,100).view(-1,1)
y_eval = model(X_eval).view(-1,1)

### PLOTING ###
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.scatter(X,y, label='test data')
plt.plot(X_eval.detach().numpy(), y_eval.detach().numpy(), color='r', label='prediction')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression : Evaluation')
plt.legend()

plt.subplot(1,2,2)
plt.plot(COST, 'o-')
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.title('Linear Regression : Cost')

plt.tight_layout()
plt.show()

# [Optional] Polynomial Regression

* Approximate the equation : $y = x^2 + x - 6$
* Above problem called "**Polynomial regression**" with the degree 2
* Slove the polynomial problem as a linear problem, $$y = x_2 + x_1 - 6; x_2 = x^2, x_1 = x$$

In [ ]:
class PolynomialDatasetDemo(Dataset):
  def __init__(self, lenght=100):
    self.len = lenght
    self.X = 8*torch.rand(lenght) - 4
    rand = 2*torch.rand_like(self.X) - 1
    self.y = self.X*self.X + self.X - 6 + 0.5*rand
  
  def __getitem__(self, index):
    sample = self.X[index], self.y[index]
    return sample
  
  def __len__(self):
    return self.len

In [ ]:
mydata_demo = PolynomialDatasetDemo()
X, y = mydata_demo[:]
print(X.shape, y.shape)
plt.scatter(X,y)
plt.show()

Practice : <b>WHY?</b>

<!-- import numpy as np

np.random.seed(111)
X = np.random.uniform(-4.0, 4.0, 10)
y = X*X + X - 6
target = np.arange(-4.0, 4.0, 0.01)
plt.plot(target, target*target + target - 6, '-r', label = 'target')
plt.scatter(X, y, label='training data', marker='o')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

# Generate the training dataset\
X1, X2 = X, X*X
X_train = np.concatenate((X1.reshape((-1,1)), X2.reshape((-1,1))), axis=1)
y_train = y

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xx, yy = np.meshgrid(np.arange(-4.0, 4.0, 0.1), np.arange(0.0, 16.0, 0.1))
zz = xx + yy - 6

ax.scatter(X_train[:,0], X_train[:,1], y_train, c='b', marker='o')
ax.plot_wireframe(xx, yy, zz, alpha=0.3, color='r')

ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y')

plt.show() -->

## Construct dataset

In [ ]:
class PolynomialDataset(Dataset):
  def __init__(self, lenght=100):
    self.len = lenght
    self.t = 8*torch.rand(lenght) - 4
    self.x1 = self.t*self.t
    self.x2 = self.t
    self.X = torch.cat((self.x1.view(-1,1), self.x2.view(-1,1)), dim=1) 
    rand = 2*torch.rand_like(self.t) - 1
    self.y = self.x1 + self.x2 - 6 + 0.5*rand
  
  def __getitem__(self, index):
    sample = self.X[index], self.y[index]
    return sample
  
  def __len__(self):
    return self.len

In [ ]:
mydata = PolynomialDataset()
X, y = mydata[:]
print(X.shape, y.shape)

In [ ]:
plt.scatter(mydata.t,y)
plt.show()

In [ ]:
trainloader = DataLoader(dataset=mydata, batch_size=1)

## Construct model

In [ ]:
model = LinearRegression(X.shape[-1],1,True)
model

In [ ]:
from torchsummary import summary
summary(model, X.view(-1,2).shape)

## Loss & Hyperparameters setting

In [ ]:
loss_fn = nn.MSELoss()
epochs = 10
learning_rate = .01
COST = []

## Construct optimizer

In [ ]:
optimizer = SGD(model.parameters(), lr=learning_rate)
optimizer

## Training model

In [ ]:
start_time = time.time()
for epoch in range(epochs):
  losses_ = []
  for X, y in trainloader:
    X, y = X.view(-1,2), y.view(-1,1)
    model.train()
    #### Compute outputs ####
    output = model(X)
    
    #### Compute Loss ####
    loss  = loss_fn(output, y)
    losses_.append(loss.item())

    #### Compute gradients ####
    optimizer.zero_grad()
    loss.backward()
    
    #### Update parameters ####  
    optimizer.step()
  
  COST.append(sum(losses_)/len(losses_))

  ### Logging ###
  print('Epoch: %03d/%03d' % ((epoch + 1), epochs), end="")
  print(' | Loss: %.3f \n' % COST[-1], end="")

print('weight: %s' % model.linear.weight.detach())
if model.linear.bias:
  print('bias: %s' % model.linear.bias.item())
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

## Evaluation model

In [ ]:
model.eval()
t = torch.linspace(-4,4,100)
X_eval = torch.cat(((t*t).view(-1,1), t.view(-1,1)), dim=1) 
y_eval = model(X_eval).view(-1,1)
y = t*t + t - 6

### PLOTING ###
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.plot(t.numpy(), y_eval.detach().numpy(), color='r', label='prediction')
plt.plot(t.numpy(), y.numpy(), color='g', label='target')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Polynomial Regression : Evaluation')
plt.legend()

plt.subplot(1,2,2)
plt.plot(COST, 'o-')
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.title('Polynomial Regression : Cost')

plt.tight_layout()
plt.show()